## Business Problem

Restaurants play a major role in the business, intellectual, social and artistic life of a society. To get into a restaurant business, one need to have a proper business plan and background analysis to seek investors attention. For a restaurant business to be in a good shape, customer ratings play a key role. In today's world, web and apps allow customers to efficiently express their views and feedback on the restaurants to a large audience. This is valuable as one can get in-depth information regarding a restaurant from other customers for a very low efforts and search costs. It is always better to have a thorough analysis of the market before getiing into the business.

We are assuming ourselves as a consulting firm who would provide valuable insights and analysis to the individuals who are planning to start a new restaurant in Banglore city. In the below analysis we have determined various factors that plays role in deciding the customer ratings for a restaurant. We also built predictive models that helps in determining a restaurants success(customer ratings) based on the business plan.

For this analysis, we have used an open source dataset scrapped from Zomato available in Kaggle.

In [2]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('PROJECT').getOrCreate()

In [5]:
#importing data to a pandas dataframe
import pandas as pd
df = pd.read_csv("/dbfs/FileStore/tables/zomato.csv")

In [6]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# As reading data directly into spark dataframe is causing data mismatch for various rows, 
# so we have first imported using pandas and then converted it into spark dataframe.

In [7]:
df = spark.createDataFrame(df)

In [9]:
df.printSchema()

root
-- url: string (nullable = true)
-- address: string (nullable = true)
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: string (nullable = true)
-- votes: long (nullable = true)
-- phone: string (nullable = true)
-- location: string (nullable = true)
-- rest_type: string (nullable = true)
-- dish_liked: string (nullable = true)
-- cuisines: string (nullable = true)
-- approx_cost(for two people): string (nullable = true)
-- reviews_list: string (nullable = true)
-- menu_item: string (nullable = true)
-- listed_in(type): string (nullable = true)
-- listed_in(city): string (nullable = true)

In [10]:
df.count()

Out[163]: 51717

In [11]:
df.dropna()

Out[164]: DataFrame[url: string, address: string, name: string, online_order: string, book_table: string, rate: string, votes: bigint, phone: string, location: string, rest_type: string, dish_liked: string, cuisines: string, approx_cost(for two people): string, reviews_list: string, menu_item: string, listed_in(type): string, listed_in(city): string]

In [12]:
df.columns

Out[165]: ['url',
 'address',
 'name',
 'online_order',
 'book_table',
 'rate',
 'votes',
 'phone',
 'location',
 'rest_type',
 'dish_liked',
 'cuisines',
 'approx_cost(for two people)',
 'reviews_list',
 'menu_item',
 'listed_in(type)',
 'listed_in(city)']

In [13]:
# Rating column has attributes in the fraction form (4/5,3/5..). We only need numerator part of it for the analysis.
# Thus we are splitting it accordingly and creating a new column named ratings in this case.
import pyspark.sql.functions as f

split_col = f.split(df['rate'], '/')
df = df.withColumn('ratings', split_col.getItem(0))

In [14]:
# As low number of votes indicates that the reviews are not authentic. 
# So, we are considering the data points for which the vote count is greater than 150.

df1=df.filter("votes > 150")

In [15]:
# Here we have included two new features to the existing data frame.
df1 = df1.withColumn('CuisinesCount', f.size(f.split(f.col('cuisines'), ',')))
df1 = df1.withColumn('dish_liked_Count', f.size(f.split(f.col('dish_liked'), ',')))

df1 = df1.withColumnRenamed('listed_in(city)','listed_in_city')
df1 = df1.withColumnRenamed( 'approx_cost(for two people)', 'approx_cost')
df1 = df1.withColumnRenamed('listed_in(type)','listed_in_type')

from pyspark.sql.types import IntegerType
df1 = df1.withColumn("approx_cost", df1["approx_cost"].cast(IntegerType()))
df1 = df1.withColumn("ratings", df1["ratings"].cast('float'))

display(df1)

url address name online_order book_table rate votes phone location rest_type dish_liked cuisines approx_cost reviews_list menu_item listed_in_type listed_in_city ratings CuisinesCount dish_liked_Count https://www.zomato.com/bangalore/jalsa-banashankari?context=eyJzZSI6eyJlIjpbNTg2OTQsIjE4Mzc1NDc0IiwiNTkwOTAiLCIxODM4Mjk0NCIsIjE4MjI0Njc2IiwiNTkyODkiLCIxODM3MzM4NiJdLCJ0IjoiUmVzdGF1cmFudHMgaW4gQmFuYXNoYW5rYXJpIHNlcnZpbmcgQnVmZmV0In19 942, 21st Main Road, 2nd Stage, Banashankari, Bangalore Jalsa Yes Yes 4.1/5 775 080 42297555
+91 9743772233 Banashankari Casual Dining Pasta, Lunch Buffet, Masala Papad, Paneer Lajawab, Tomato Shorba, Dum Biryani, Sweet Corn Soup North Indian, Mughlai, Chinese 800 [('Rated 4.0', 'RATED\n A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.'), ('Rated 4.0', 'RATED\n I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\nCheers'), ('Rated 2.0', 'RATED\n Its a restaurant near to Banashankari BDA. Me along with few of my office friends visited to have buffet but unfortunately they only provide veg buffet. On inquiring they said this place is mostly visited by vegetarians. Anyways we ordered ala carte items which took ages to come. Food was ok ok. Definitely not visiting anymore.'), ('Rated 4.0', 'RATED\n We went here on a weekend and one of us had the buffet while two of us took Ala Carte. Firstly the ambience and service of this place is great! The buffet had a lot of items and the good was good. We had a Pumpkin Halwa intm the dessert which was amazing. Must try! The kulchas are great here. Cheers!'), ('Rated 5.0', 'RATED\n The best thing about the place is itÃ\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x92s ambiance. Second best thing was yummy ? food. We try buffet and buffet food was not disappointed us.\nTest ?. ?? ?? ?? ?? ??\nQuality ?. ??????????.\nService: Staff was very professional and friendly.\n\nOverall experience was excellent.\n\nsubirmajumder85.wixsite.com'), ('Rated 5.0', 'RATED\n Great food and pleasant ambience. Expensive but Coll place to chill and relax......\n\nService is really very very good and friendly staff...\n\nFood : 5/5\nService : 5/5\nAmbience :5/5\nOverall :5/5'), ('Rated 4.0', 'RATED\n Good ambience with tasty food.\nCheese chilli paratha with Bhutta palak methi curry is a good combo.\nLemon Chicken in the starters is a must try item.\nEgg fried rice was also quite tasty.\nIn the mocktails, recommend "Alice in Junoon". Do not miss it.'), ('Rated 4.0', 'RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x92t go wrong with Jalsa. Never been a fan of their buffet and thus always order alacarteÃ\x83Ã\x83Ã\x82Ã\x82Ã\x83Ã\x82Ã\x82Ã\x92. Service at times can be on the slower side but food is worth the wait.'), ('Rated 5.0', 'RATED\n Overdelighted by the service and food provided at this place. A royal and ethnic atmosphere builds a strong essence of being in India and also the quality and taste of food is truly authentic. I would totally recommend to visit this place once.'), ('Rated 4.0', 'RATED\n The place is nice and comfortable. Food wise all jalea outlets maintain a good standard. The soya chaap was a standout dish. Clearly one of trademark dish as per me and a must try.\n\nThe only concern is the parking. It very congested and limited to just 5cars. The basement parking is very steep and makes it cumbersome'), ('Rated 4.0', 'RATED\n The place is nice and comfo

In [16]:
df1.printSchema()


root
-- url: string (nullable = true)
-- address: string (nullable = true)
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: string (nullable = true)
-- votes: long (nullable = true)
-- phone: string (nullable = true)
-- location: string (nullable = true)
-- rest_type: string (nullable = true)
-- dish_liked: string (nullable = true)
-- cuisines: string (nullable = true)
-- approx_cost: integer (nullable = true)
-- reviews_list: string (nullable = true)
-- menu_item: string (nullable = true)
-- listed_in_type: string (nullable = true)
-- listed_in_city: string (nullable = true)
-- ratings: float (nullable = true)
-- CuisinesCount: integer (nullable = false)
-- dish_liked_Count: integer (nullable = false)

In [17]:
# Creating a view zomato, with the selected columns as features.
df1.select([ 'name','online_order', 'book_table', 'votes', 'location', 'rest_type', 'cuisines', 'approx_cost', 'listed_in_type','ratings','CuisinesCount','dish_liked_Count','listed_in_city']).createOrReplaceTempView('zomato')

In [18]:
df1.count()

Out[171]: 15084

#### Analysis

#### 1. Does online order availability in the restaurants decide the ratings?

In [21]:
%sql
select online_order,avg(ratings) as average_rating from zomato group by online_order 


online_order,average_rating
No,4.094441835752795
Yes,3.977004994571507


##### From the above two averages, we see that they both are almost equal. So we can say that online order does not play an important role in deciding the ratings of the restaurants. Observing closely, we see that people coming to restaurants give slightly higher ratings than those who order online.

#### 2. Does online table booking option availability in the restaurants decide the ratings?

In [24]:
%sql
select book_table,avg(ratings) as average_rating from zomato group by book_table 

book_table,average_rating
No,3.9090531950086946
Yes,4.190330311785239


##### From the above two averages, we see that there is almost 0.3 rating higher for the restaurants with online table booking option availability.

#### 3a. Top five locations for restaurants to avail better ratings

In [27]:
%sql
select location,avg(ratings) as average_rating from zomato group by location order by average_rating desc limit 5 

location,average_rating
City Market,4.399999936421712
Lavelle Road,4.294812734944676
Hennur,4.235294173745548
Koramangala 5th Block,4.218389154924976
Vasanth Nagar,4.2156862230861885


##### The above five locations are the most favourable for restaurants to come up with better customer satisfaction.

#### 3b. Top five restaurants having better ratings

In [30]:
%sql
select name,avg(ratings) as average_rating from zomato group by name order by average_rating desc limit 5 

name,average_rating
Asia Kitchen By Mainland China,4.900000095367432
Byg Brewski Brewing Company,4.900000095367432
SantÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Spa Cuisine,4.900000095367432
Punjab Grill,4.871428694043841
Belgian Waffle Factory,4.844827734190842


##### The above listed five are the restaurants to come up with better customer satisfaction.

#### 3c. Are the top five rated restaurants located in above listed top 5 favourable locations with better customer satisfaction?

In [33]:
%sql
select name,avg(ratings) as avg_rating,location from zomato group by name,location order by avg_rating desc limit 5

name,avg_rating,location
Flechazo,4.900000095367432,Whitefield
Byg Brewski Brewing Company,4.900000095367432,Sarjapur Road
Belgian Waffle Factory,4.900000095367432,Brigade Road
Asia Kitchen By Mainland China,4.900000095367432,Koramangala 5th Block
Milano Ice Cream,4.900000095367432,Indiranagar


##### The above result gives some interesting findings. The top five rated restaurants are not in the top five favourable locations. So location is not the only factor that decides the ratings.

#### 4. Will the number of cuisines have an impact on the customer satisfaction rate (Are customers satisfied with restaurants concentrating on single cuisine or inclined towards multi-cuisines)?

In [36]:
%sql

select CuisinesCount, avg(ratings) as average_rating from zomato group by CuisinesCount order by average_rating desc

CuisinesCount,average_rating
6,4.172398172352649
7,4.1575439034846795
5,4.104934194958524
4,4.046109984178498
-1,4.0
3,3.9934663217263924
2,3.967367345019812
8,3.965833322207133
1,3.9647505932352605


##### From the above analysis, we can say that muti-cuisine restaurants are more liked comapred to single-cuisine restaurants. 

It is interesting to know that, to start a multi-cuisine restaurants it is ideal to have 5 or 6 cuisines. However, if planning for other number of cuisines, the rating is almost same for 1 to 4 cuisines. So, its better to go with single cuisine.

#### Model predictions

##### Because our dependent variable is a continuous varible ranging from zero to five, we cannot use models like logistic regression here. For such continuous varibles with number of inputs, it will be a good example of a regression model problem.

In [40]:
# Selecting the dependent and the independent variables that are identified as most useful attributes to estimate duration

data=df1.select(['online_order','book_table','dish_liked_count','CuisinesCount','listed_in_city','listed_in_type','approx_cost','ratings'])

In [41]:
data.count()

Out[173]: 15084

In [42]:
data=data.dropna()

In [43]:
data.count()

Out[175]: 9977

In [44]:
# Create a 70-30 train test split

train_data,test_data=data.randomSplit([0.7,0.3])

#### Building Random Forest Model:

In [46]:
data.describe().show()

+-------+------------+----------+-----------------+------------------+--------------+--------------+------------------+------------------+
summary|online_order|book_table| dish_liked_count| CuisinesCount|listed_in_city|listed_in_type| approx_cost| ratings|
+-------+------------+----------+-----------------+------------------+--------------+--------------+------------------+------------------+
 count| 9977| 9977| 9977| 9977| 9977| 9977| 9977| 9977|
 mean| null| null|6.114563496040894| 2.915806354615616| null| null| 551.8893454946377| 3.920106237489307|
 stddev| null| null|1.839490781604514|1.4126986429105632| null| null|193.52869719935035|0.4119830337048238|
 min| No| No| -1| 1| BTM| Buffet| 40| 2.0|
 max| Yes| Yes| 7| 8| Whitefield| Pubs and bars| 950| 4.9|
+-------+------------+----------+-----------------+------------------+--------------+--------------+------------------+------------------+

In [47]:
#Created string indexes to a categorical variable in order to convert each category into a specific index(neumerical value)
from pyspark.ml.feature import StringIndexer
online_order_indexer = StringIndexer(inputCol='online_order',outputCol='online_order_index',handleInvalid='keep')
online_order_indexer_df = online_order_indexer.fit(data).transform(data)
online_order_indexer_df.show()

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 3.8| 0.0|
 No| No| 2| 2| Banashankari| Buffet| 600| 3.8| 1.0|
 Yes| No| 7| 1| Banashankari| Buffet| 600| 3.8| 0.0|
 Yes| Yes| 7| 3| Banashankari| Cafes| 600| 4.6| 0.0|
 Yes| No| 7| 3| Banashankari| Cafes| 700| 4.0| 0.0|
 Yes| No| 7| 5| Banashankari| Cafes| 550| 4.2| 0.0|
 Yes| No| 7| 1| Banashankari| Cafes| 500| 4.1| 0.0|
 Yes| Yes| 7| 4| Banashankari| Cafes| 500| 4.2| 0.0|
 No| No| 7| 2| Banashankari| Cafes| 450| 4.0| 1.0|
 Yes| No| 7| 3| Banashankari| Cafes| 800| 3.8| 0.0|
 Yes| No| 2| 4| Banashankari| Cafes| 500| 3.7| 0.0|
 Yes| No| 7| 2| Banashankari| Cafes| 600| 4.0| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 500| 2.8| 0.0|
 Yes| No| 5| 1| Banashankari| Delivery| 750| 3.9| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 500| 4.2| 0.0|
 Yes| No| 7| 1| Banashankari| Delivery| 650| 3.9| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 600| 4.2| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 750| 4.4| 0.0|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+
only showing top 20 rows

In [48]:
#Created string indexes to a categorical variable in order to convert each category into a specific index(neumerical value)
from pyspark.ml.feature import StringIndexer
book_table_indexer = StringIndexer(inputCol='book_table',outputCol='book_table_index',handleInvalid='keep')
book_table_indexer_df = book_table_indexer.fit(online_order_indexer_df).transform(online_order_indexer_df)
book_table_indexer_df.show()

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|book_table_index|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 1.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 0.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 3.8| 0.0| 0.0|
 No| No| 2| 2| Banashankari| Buffet| 600| 3.8| 1.0| 0.0|
 Yes| No| 7| 1| Banashankari| Buffet| 600| 3.8| 0.0| 0.0|
 Yes| Yes| 7| 3| Banashankari| Cafes| 600| 4.6| 0.0| 1.0|
 Yes| No| 7| 3| Banashankari| Cafes| 700| 4.0| 0.0| 0.0|
 Yes| No| 7| 5| Banashankari| Cafes| 550| 4.2| 0.0| 0.0|
 Yes| No| 7| 1| Banashankari| Cafes| 500| 4.1| 0.0| 0.0|
 Yes| Yes| 7| 4| Banashankari| Cafes| 500| 4.2| 0.0| 1.0|
 No| No| 7| 2| Banashankari| Cafes| 450| 4.0| 1.0| 0.0|
 Yes| No| 7| 3| Banashankari| Cafes| 800| 3.8| 0.0| 0.0|
 Yes| No| 2| 4| Banashankari| Cafes| 500| 3.7| 0.0| 0.0|
 Yes| No| 7| 2| Banashankari| Cafes| 600| 4.0| 0.0| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 500| 2.8| 0.0| 0.0|
 Yes| No| 5| 1| Banashankari| Delivery| 750| 3.9| 0.0| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 500| 4.2| 0.0| 0.0|
 Yes| No| 7| 1| Banashankari| Delivery| 650| 3.9| 0.0| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 600| 4.2| 0.0| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 750| 4.4| 0.0| 0.0|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+
only showing top 20 rows

In [49]:
#Created string indexes to a categorical variable in order to convert each category into a specific index(neumerical value)
from pyspark.ml.feature import StringIndexer
listed_in_city_indexer = StringIndexer(inputCol='listed_in_city',outputCol='listed_in_city_index',handleInvalid='keep')
listed_in_city_indexer_df = listed_in_city_indexer.fit(book_table_indexer_df).transform(book_table_indexer_df)
listed_in_city_indexer_df.show()

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|book_table_index|listed_in_city_index|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 1.0| 23.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 3.8| 0.0| 0.0| 23.0|
 No| No| 2| 2| Banashankari| Buffet| 600| 3.8| 1.0| 0.0| 23.0|
 Yes| No| 7| 1| Banashankari| Buffet| 600| 3.8| 0.0| 0.0| 23.0|
 Yes| Yes| 7| 3| Banashankari| Cafes| 600| 4.6| 0.0| 1.0| 23.0|
 Yes| No| 7| 3| Banashankari| Cafes| 700| 4.0| 0.0| 0.0| 23.0|
 Yes| No| 7| 5| Banashankari| Cafes| 550| 4.2| 0.0| 0.0| 23.0|
 Yes| No| 7| 1| Banashankari| Cafes| 500| 4.1| 0.0| 0.0| 23.0|
 Yes| Yes| 7| 4| Banashankari| Cafes| 500| 4.2| 0.0| 1.0| 23.0|
 No| No| 7| 2| Banashankari| Cafes| 450| 4.0| 1.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Cafes| 800| 3.8| 0.0| 0.0| 23.0|
 Yes| No| 2| 4| Banashankari| Cafes| 500| 3.7| 0.0| 0.0| 23.0|
 Yes| No| 7| 2| Banashankari| Cafes| 600| 4.0| 0.0| 0.0| 23.0|
 Yes| No| 7| 4| Banashankari| Delivery| 500| 2.8| 0.0| 0.0| 23.0|
 Yes| No| 5| 1| Banashankari| Delivery| 750| 3.9| 0.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Delivery| 500| 4.2| 0.0| 0.0| 23.0|
 Yes| No| 7| 1| Banashankari| Delivery| 650| 3.9| 0.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Delivery| 600| 4.2| 0.0| 0.0| 23.0|
 Yes| No| 7| 4| Banashankari| Delivery| 750| 4.4| 0.0| 0.0| 23.0|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+
only showing top 20 rows

In [50]:
# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=['online_order_index','book_table_index','listed_in_city_index',
                                       'dish_liked_count','CuisinesCount','approx_cost'],
                            outputCol="features")
output = assembler.transform(listed_in_city_indexer_df)
output.show(2)

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+--------------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|book_table_index|listed_in_city_index| features|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+--------------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 1.0| 23.0|[0.0,1.0,23.0,7.0...|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 0.0| 23.0|[0.0,0.0,23.0,7.0...|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+--------------------+
only showing top 2 rows

In [51]:
#Applying features and lables to provide the same to model.
df_classifier = output.selectExpr("features as features","ratings as label")
df_classifier.show(2)

+--------------------+-----+
 features|label|
+--------------------+-----+
[0.0,1.0,23.0,7.0...| 4.1|
[0.0,0.0,23.0,7.0...| 4.1|
+--------------------+-----+
only showing top 2 rows

In [52]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
#from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.



featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=5).fit(df_classifier)

In [53]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_classifier.randomSplit([0.7, 0.3])

In [54]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

In [55]:
# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

In [56]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [57]:
predictions = model.transform(testData)

In [58]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
3.6972870561851146| 4.0|[0.0,0.0,9.0,-1.0...|
 3.59753293113357| 3.1|[0.0,0.0,9.0,-1.0...|
3.7029975420344514| 3.7|[0.0,0.0,9.0,1.0,...|
 3.665873672626455| 3.7|[0.0,0.0,9.0,1.0,...|
3.6371709827382817| 3.9|[0.0,0.0,9.0,1.0,...|
+------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.358753
RandomForestRegressionModel (uid=RandomForestRegressor_47b7416d01ce) with 20 trees

##### For the Random forest regression model, the Root Mean Squared Error 0.37 explains that the standard deviation of the residuals is low. It also indicates that most of the data points lie closely to the regression line.

### Decision Tree:

In [61]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [62]:
# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

In [63]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)


In [64]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
3.6128205397190194| 4.0|[0.0,0.0,9.0,-1.0...|
3.5078176007596986| 3.1|[0.0,0.0,9.0,-1.0...|
3.6128205397190194| 3.7|[0.0,0.0,9.0,1.0,...|
3.5078176007596986| 3.7|[0.0,0.0,9.0,1.0,...|
3.5078176007596986| 3.9|[0.0,0.0,9.0,1.0,...|
+------------------+-----+--------------------+
only showing top 5 rows

In [65]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
treeModel = model.stages[1]
# summary only
print(treeModel)

Root Mean Squared Error (RMSE) on test data = 0.370854
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_e47371596568) of depth 5 with 61 nodes

##### For the Decision Tree model, the Root Mean Squared Error 0.377 explains that the standard deviation of the residuals is low. It also indicates that most of the data points lie closely to the regression line.

### Linear Regression

In [68]:
from pyspark.ml.regression import LinearRegression

In [69]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(trainingData)

In [70]:
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 3.916869185599708

In [71]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 1
objectiveHistory: [0.5]
+--------------------+
 residuals|
+--------------------+
-0.01686909023227...|
 -0.4168691855997082|
 -0.4168691855997082|
-0.21686913791599238|
 -0.7168691379159924|
-0.11686923328342402|
-0.11686923328342402|
-0.21686913791599238|
-0.11686923328342402|
-0.11686923328342402|
 0.0831308144002918|
 0.0831308144002918|
 0.0831308144002918|
-0.01686909023227...|
 -0.9168691855997082|
 -0.9168691855997082|
-0.01686909023227...|
-0.31686928096713984|
 -0.9168691855997082|
 0.3831310051351551|
+--------------------+
only showing top 20 rows

RMSE: 0.416351
r2: -0.000000

##### For the Linear regression model, the Root Mean Squared Error 0.413 explains that the standard deviation of the residuals is low. It also indicates that most of the data points lie closely to the regression line.

##### From the above three models evaluations, we can see that the Root Mean Square Error of these models are almost same, as we have choosen the best possible models for this kind of data. On closely observing the model performance, we see that Random Forest model performed better than other two models with lower RMSE.

##### As a consulting firm, we are now in a position to provide a deep analysis on which location, number of cusinies, price range and theme of restaurent to choose to any prospective client who wants to get into the restaurant business in city of Banglore. Our model is good enough to predict the ratings which gives an idea of how the business performed based on various factors.